<a href="https://colab.research.google.com/github/Grizzzley/Pytap-NN-model-creation/blob/main/Pytup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение модели

Мы обучим простой классификатор изображений на PyTorch с помощью библиотеки [timm](https://timm.fast.ai).

Сначала установим необходимые пакеты. Не забудьте подключить GPU в Colab ("Среда выполнения" -> "Сменить среду выполнения" -> "Аппаратный ускоритель" -> "GPU").

In [ ]:
%pip install opencv-python timm wget crowd-kit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 33.4 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 4.7 MB 54.8 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 596 kB 71.9 MB/s 
     |████████████████████████████████| 6.6 MB 56.8 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=bc06159bef86c257dcaf865061ee571f1ceaeb1338a397902439db2d2601b715
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Теперь скачаем наши изображения.

In [ ]:
!wget -O imgs.zip http://tlk.s3.yandex.net/tutorials/pytup/imgs.zip
!unzip imgs.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: imgs/104243.jpg         
  inflating: imgs/71694.jpg          
  inflating: imgs/60941.jpg          
  inflating: imgs/46038.jpg          
  inflating: imgs/5056.jpg           
  inflating: imgs/142216.jpg         
  inflating: imgs/18622.jpg          
  inflating: imgs/26914.jpg          
  inflating: imgs/17511.jpg          
  inflating: imgs/124193.jpg         
  inflating: imgs/51544.jpg          
  inflating: imgs/85073.jpg          
  inflating: imgs/132929.jpg         
  inflating: imgs/152005.jpg         
  inflating: imgs/114050.jpg         
  inflating: imgs/61487.jpg          
  inflating: imgs/50882.jpg          
  inflating: imgs/95260.jpg          
  inflating: imgs/134380.jpg         
  inflating: imgs/2739.jpg           
  inflating: imgs/41757.jpg          
  inflating: imgs/145579.jpg         
  inflating: imgs/39834.jpg          
  inflating: imgs/59118.jpg          
  inflating: imgs/11

Импортируем все нужное

In [ ]:
import wget
import pandas as pd  # для работы с данными
import os  # чтобы перемещать файлы
from tqdm.auto import tqdm  # показывать progress bar
import shutil  # тоже перемещать файлы
import timm  # здесь реализованны предобученые computer vision модели
from timm.data.dataset_factory import create_dataset
from timm.data.transforms_factory import create_transform
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn
import torch

from crowdkit.aggregation import DawidSkene

Нам нужно по результатам разметки разложить картинки по папочкам следующим образом: в папке `dataset` создать две поддиректории `male` и `female`. В первую нужно класть те изображения, которые исполнители посчитали изображениями мужчин, во вторую — женщин.



In [ ]:
os.mkdir('dataset')
os.mkdir('dataset/male')
os.mkdir('dataset/female')

Считываем данные.

In [ ]:
df = pd.read_csv('assignments.tsv', sep='\t')
df.head()

,INPUT:image,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started
0,http://tlk.s3.yandex.net/tutorials/pydup/13862...,female,NaN,NaN,NaN,https://toloka.yandex.ru/task/34836386/0002138...,0002138fa2--62f3d7c770c69b30c0be7500,0002138fa2--62f520b8b956933143d38d89,49f36613960424e7811aad3cb5c3688b,APPROVED,2022-08-11T15:31:04.588
1,http://tlk.s3.yandex.net/tutorials/pydup/30783...,female,NaN,NaN,NaN,https://toloka.yandex.ru/task/34836386/0002138...,0002138fa2--62f3d76570c69b30c0be6794,0002138fa2--62f520b8b956933143d38d89,49f36613960424e7811aad3cb5c3688b,APPROVED,2022-08-11T15:31:04.588
2,http://tlk.s3.yandex.net/tutorials/pydup/16085...,female,female,NaN,NaN,https://toloka.yandex.ru/task/34836386/0002138...,0002138fa2--62f3d76570c69b30c0be66e0,0002138fa2--62f520b8b956933143d38d89,49f36613960424e7811aad3cb5c3688b,APPROVED,2022-08-11T15:31:04.588
3,http://tlk.s3.yandex.net/tutorials/pydup/91985...,female,NaN,NaN,NaN,https://toloka.yandex.ru/task/34836386/0002138...,0002138fa2--62f3d78670c69b30c0be6ca2,0002138fa2--62f520b8b956933143d38d89,49f36613960424e7811aad3cb5c3688b,APPROVED,2022-08-11T15:31:04.588
4,http://tlk.s3.yandex.net/tutorials/pydup/3036.jpg,female,NaN,NaN,NaN,https://toloka.yandex.ru/task/34836386/0002138...,0002138fa2--62f3d99a70c69b30c0beb91b,0002138fa2--62f520b8b956933143d38d89,49f36613960424e7811aad3cb5c3688b,APPROVED,2022-08-11T15:31:04.588


Как мы видим, в результатах разметки много разной информаци. Нас интересуют следующие колонки:
* **INPUT:image** — ссылка на картинку
* **OUTPUT:result** — лейбл, который поставил исполнитель
* **ASSIGNMENT:worker_id** — ID исполнителя

Теперь нам нужно для каждой картинки из разметки в перекрытии 3 получить едиственный лейбл. Для этого воспользуемся агрегацией через алгоритм Дэвида-Скина, который реализован в библиотеке [Crowd-Kit](https://github.com/Toloka/crowd-kit).

Для этого, нужно переименовать колонки выше в `task`, `label` и `worker`.

In [ ]:
df = df[['INPUT:image', 'OUTPUT:result', 'ASSIGNMENT:worker_id']]
df.columns = ['task', 'label', 'worker']

result = DawidSkene().fit_predict(df)

Посмотрим на результаты.

In [ ]:
result

task
http://tlk.s3.yandex.net/tutorials/pydup/138624.jpg    female
http://tlk.s3.yandex.net/tutorials/pydup/30783.jpg     female
http://tlk.s3.yandex.net/tutorials/pydup/160856.jpg    female
http://tlk.s3.yandex.net/tutorials/pydup/91985.jpg     female
http://tlk.s3.yandex.net/tutorials/pydup/3036.jpg      female
                                                        ...  
http://tlk.s3.yandex.net/tutorials/pydup/84153.jpg     female
http://tlk.s3.yandex.net/tutorials/pydup/107205.jpg    female
http://tlk.s3.yandex.net/tutorials/pydup/57491.jpg       male
http://tlk.s3.yandex.net/tutorials/pydup/25614.jpg       male
http://tlk.s3.yandex.net/tutorials/pydup/66738.jpg       male
Name: agg_label, Length: 8891, dtype: object

Теперь разложим картинки по папочкам!

In [ ]:
for img_url, label in tqdm(result.iteritems(), total=len(result)):
  img = img_url.split('/')[-1]
  if label == 'male' or label == 'female':
    shutil.copy(f'imgs/{img}', f'dataset/{label}/{img}')

  0%|          | 0/8891 [00:00<?, ?it/s]

Создадим датасет.

In [ ]:
ds = create_dataset(name='', root='dataset', transform=create_transform(128))

Теперь скачаем предобученную на ImageNet модель. Возьмем ResNet50

In [ ]:
model = timm.create_model('resnet50d', pretrained=True, num_classes=2)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50d_ra2-464e36ba.pth" to /root/.cache/torch/hub/checkpoints/resnet50d_ra2-464e36ba.pth


Поделим выборку на обучающую и валидационную.

In [ ]:
train, val = train_test_split(ds, test_size=0.2)
train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)

Учить будем на GPU

In [ ]:
model = model.cuda()

Создаем оптимизатор, Adam — стандартный выбор.

In [ ]:
optim = Adam(model.parameters(), lr=1e-3)

Запускаем обучение, каждую эпоху будем печатать долю правильных предсказаний (accuracy).

In [ ]:
criterion = nn.CrossEntropyLoss()
for epoch in range(10):
  print(f'==Epoch {epoch}==')
  train_correct = 0
  train_total = 0
  for X, y in tqdm(train_loader):
    optim.zero_grad()
    preds = model(X.cuda())
    loss = criterion(preds, y.cuda())
    loss.backward()
    optim.step()
    labels = torch.argmax(preds, axis=1)
    train_total += len(y)
    train_correct += torch.sum(labels == y.cuda()).item()
  print(f'Train acc = {train_correct / train_total}')
  val_correct = 0
  val_total = 0
  with torch.no_grad():
    for X, y in tqdm(val_loader):
      preds = model(X.cuda())
      labels = torch.argmax(preds, axis=1)
      val_total += len(y)
      val_correct += torch.sum(labels == y.cuda()).item()
  print(f'Val acc = {val_correct / val_total}')

==Epoch 0==


  0%|          | 0/219 [00:00<?, ?it/s]

Train acc = 0.9074285714285715


  0%|          | 0/55 [00:00<?, ?it/s]

Val acc = 0.9354285714285714
==Epoch 1==


...

Train acc = 0.9858571428571429


  0%|          | 0/55 [00:00<?, ?it/s]

Val acc = 0.9422857142857143
==Epoch 6==


  0%|          | 0/219 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), 'gender_classification.pth')

In [ ]:
import skimage.io
import timm
import torch
from torch.nn.functional import softmax
from fastapi import FastAPI
from torchvision.transforms import Resize
import numpy as np
from fastapi import File, UploadFile, Form
from fastapi.responses import HTMLResponse
import cv2
import uvicorn

app = FastAPI()

model = timm.create_model('resnet50d', pretrained=False, num_classes=2)
model.load_state_dict(torch.load('gender_classification.pth', map_location=torch.device('cpu')))
idx_to_class = {1: 'male', 0: 'female'}
transform = Resize((128, 128))


def apply_model(model, image_numpy):
    image_torch = transform(torch.tensor(image_numpy).float().permute(2, 0, 1).unsqueeze(0))

    probs = softmax(model(image_torch), dim=-1).cpu().detach().numpy()[0]
    label = np.argmax(probs)

    return {'label': idx_to_class[label], 'prob': float(probs[label])}


def classify_image(model, image_url):
    image_numpy = skimage.io.imread(image_url) / 255.
    return apply_model(model, image_numpy)


@app.get('/')
def get_root():
	return {'message': 'Welcome to the gender classification API'}


@app.get('/predict/')
async def classify_image_method(image_url: str):
	return classify_image(model, image_url)


@app.get('/predict', response_class=HTMLResponse)
async def return_html():
    with open('predict.html') as f:
        html = f.read()
    return HTMLResponse(content=html, status_code=200)


@app.post("/predict_file")
def upload(file: UploadFile = File(...)):
    try:
        contents = file.file.read()
        decoded = cv2.imdecode(np.frombuffer(contents, np.uint8), -1)[:, :, :3] / 255.
        file.file.close()
        response = apply_model(model, decoded)
        response['message'] = 'Successfuly processed file'
        return response
    except Exception as e:
        return {"message": f"Failed to predict a label for {file.filename}"}



@app.get('/predict/{image_url}')
async def classify_image_method_path(image_url: str):
	return classify_image(model, image_url)


In [ ]:
!uvicorn inference:app --workers 4 --port 9999 --host 0.0.0.0

In [ ]:
!curl -X 'GET' \
  'http://158.160.10.14:9999/predict/?image_url=http%3A%2F%2Ftlk.s3.yandex.net%2Ftutorials%2Fpydup%2F138624.jpg' \
  -H 'accept: application/json'

{"label":"female","prob":0.9992744326591492}